In [1]:
!pip install sigkerax
#!pip install tslearn
import math
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import jax
import jax.numpy as jnp
import jax.random as jr
import jax.nn as jnn
from jax import ops
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
# from tslearn.datasets import UCR_UEA_datasets
# from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pywt

from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import os
import sys


# import utility
# sys.path.append('/kaggle/usr/lib/utility')
import sigkerax
from sigkerax.sigkernel import SigKernel
#!cp -r /opt/conda/lib/python3.10/site-packages/sigkerax* /kaggle/working

# Load the metadata

Here I specify which features to use, create the targets, filter by unique eeg_id/vote combinations and also filter dataset by "best" datapoints where number of votes > 10.

In [2]:
PATH_TRAIN = '/kaggle/input/hms-harmful-brain-activity-classification/train.csv'
EEG_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
#Specify relevant features
FEATS = ['Fp1','C3','T3','O1','Fp2','C4','T4','O2']
df = pd.read_csv(PATH_TRAIN)
TARGETS = df.columns[-6:]
df_unique = df.drop_duplicates(subset=['eeg_id', 'seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'], keep='first')
df_unique[TARGETS] = df_unique[TARGETS].astype('float64')
y = df_unique[TARGETS].agg('sum', axis=1)
df_unique['sum_votes']=y
# strong samples
df_unique = df_unique.loc[df_unique['sum_votes']>10]

#unanimous samples optional

# df_unique['max_category_votes'] = df_unique[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].max(axis=1)
# df_unique = df_unique[df_unique['max_category_votes'] == df_unique['sum_votes']]
# df_unique.drop(columns=['max_category_votes'], inplace=True)

class_counts = df_unique[TARGETS].sum(axis=0)
print(class_counts)
# min_count = class_counts.min()

# # Perform undersampling
# avg_other_classes = (class_counts.sum() - class_counts['other_vote']) / (len(class_counts) - 1)

# # Set a target for the other_vote class, could be the average or another reference point
# target_other_vote = avg_other_classes

# # Calculate the current proportion of the other_vote class
# current_proportion_other = df_unique['other_vote'] / df_unique['sum_votes']

# # Determine a threshold proportion to decide which samples to keep
# # This is arbitrary and should be adjusted according to your specific needs
# # For example, keep samples where 'other_vote' is not the absolute majority
# threshold_proportion = 0.6

# # Filter out samples where 'other_vote' makes up more than the threshold proportion
# df_unique = df_unique[df_unique['other_vote'] / df_unique['sum_votes'] <= threshold_proportion]
# class_counts = df_unique[TARGETS].sum(axis=0)
# print(class_counts, len(df_unique))

# create the targets
v = df_unique[TARGETS].values
v = v / v.sum(axis=1).reshape((-1, 1))
df_unique.loc[:, TARGETS] = v.astype('float64')
len(df_unique)

seizure_vote     4793.0
lpd_vote        16669.0
gpd_vote        10971.0
lrda_vote        7007.0
grda_vote        7460.0
other_vote      43767.0
dtype: float64


/tmp/ipykernel_34/2062774691.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique[TARGETS] = df_unique[TARGETS].astype('float64')
/tmp/ipykernel_34/2062774691.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['sum_votes']=y


6233

In [24]:
df_unique.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,sum_votes
9,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0.0,0.0000,0.454545,0.000,0.090909,0.454545,11.0
11,722738444,0,0.0,999431,0,0.0,557980729,56885,LRDA,0.0,0.0625,0.000000,0.875,0.000000,0.062500,16.0
35,1202099836,0,0.0,1353070,0,0.0,3051612434,34554,Other,0.0,0.0000,0.357143,0.000,0.000000,0.642857,14.0
55,2578018731,1,20.0,2207717,1,20.0,1928228287,34153,GRDA,0.0,0.0000,0.153846,0.000,0.692308,0.153846,13.0
69,374504640,0,0.0,3452193,0,0.0,474823595,29847,GRDA,0.0,0.0000,0.166667,0.000,0.666667,0.166667,12.0


# Load batch function
A function to load the dataset in batches, applicable to the train dataset. Returns jax.numpy.Array. Also denoising function, montage tranformation, Fourier transform, slicing and masking.

In [3]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    ret=pywt.waverec(coeff, wavelet, mode='per')

    return ret

def montage(eeg_data, electrodes=FEATS):
    """
    Apply bipolar montaging to EEG data.
    
    Parameters:
    - eeg_data: NumPy array with shape (timepoints, channels), where channels correspond to electrodes.
    - electrodes: List of electrode names in the same order as the eeg_data channels.
    
    Returns:
    - montaged_data: NumPy array after applying bipolar montaging.
    """
    # Define pairs for bipolar montage based on electrode positions
    # This is a simplified example; pairs should be chosen based on electrode locations and analysis needs
    pairs = [('Fp1', 'T3'), ('T3', 'O1'), ('Fp1', 'C3'), ('C3', 'O1'),
             ('Fp2', 'C4'), ('C4', 'O2'), ('Fp2', 'T4'), ('T4', 'O2')]
    
    montaged_data = []
    for pair in pairs:
        idx1 = electrodes.index(pair[0])
        idx2 = electrodes.index(pair[1])
        # Compute the difference between the pairs
        montaged_data.append(eeg_data[:, idx1] - eeg_data[:, idx2])
    
    # Stack the montaged data to form a new array
    montaged_data = np.stack(montaged_data, axis=-1)
    
    return montaged_data

def ft(eeg_data):
    # Apply Fast Fourier Transform (FFT) to each channel
    fft_results = np.fft.rfft(eeg_data, axis=0)
    
    # Compute magnitude spectrum (amplitude)
    magnitude_spectrum = np.abs(fft_results)
    
    # Optionally, compute the power spectrum or log power spectrum for analysis
    power_spectrum = np.square(magnitude_spectrum)
    log_power_spectrum = np.log(power_spectrum + 1e-10)  # Adding epsilon to avoid log(0)
    
    return log_power_spectrum

def fft_filter(signal, fs):
    """
    Filters the EEG signal to retain important brain wave frequencies
    and reduces the size by a factor of 8.
    
    Parameters:
    - signal: The time-domain EEG signal (1D numpy array).
    - fs: Sampling frequency of the signal.
    
    Returns:
    - reduced_signal: The time-domain signal after filtering and size reduction.
    """
    # Perform FFT
    fft_result = np.fft.fft(signal)
    freq = np.fft.fftfreq(len(signal), d=1/fs)
    
    # Define brain wave frequency ranges
    delta = (0.5, 4)
    theta = (4, 8)
    alpha = (8, 12)
    beta = (12, 30)
    
    # Create a mask to retain frequencies within the defined brain wave ranges
    mask = ((np.abs(freq) >= delta[0]) & (np.abs(freq) <= delta[1])) | \
           ((np.abs(freq) >= theta[0]) & (np.abs(freq) <= theta[1])) | \
           ((np.abs(freq) >= alpha[0]) & (np.abs(freq) <= alpha[1])) | \
           ((np.abs(freq) >= beta[0]) & (np.abs(freq) <= beta[1]))
    
    # Apply mask and zero out other frequencies
    fft_result_filtered = np.zeros_like(fft_result)
    fft_result_filtered[mask] = fft_result[mask]
    
    # Optionally, reduce the size of the filtered FFT result by a factor of 8
    # This simplistic approach keeps every 8th frequency component
    reduction_factor = 2
    reduced_fft_result_filtered = fft_result_filtered[::reduction_factor]
    
    # Perform Inverse FFT to get the filtered, reduced signal back in time domain
    reduced_signal = np.fft.ifft(reduced_fft_result_filtered).real
    
    return reduced_signal[::4]

def load_data_batch(batch_size, batch_num, df= df_unique):
    start_index = batch_num * batch_size
    end_index = start_index + batch_size
    batch_df = df.iloc[start_index:end_index]
    data = []

    for row in batch_df.itertuples():
        eeg_id = getattr(row, 'eeg_id')
        offset = int(getattr(row, 'eeg_label_offset_seconds') * 200)
        eeg = pq.read_table(f"{EEG_PATH}{eeg_id}.parquet").to_pandas().iloc[offset + 4000:offset + 6000]
        eeg = eeg.loc[:, eeg.columns.isin(FEATS)].ffill().bfill().values
        
        
        #eeg = montage(eeg)
        eeg = fft_filter(eeg, 200)
        #eeg = denoise(eeg)
        #eeg = ft(eeg)
        max_val = np.abs(eeg).max(axis=0, keepdims=True)
        max_val[max_val == 0] = 1  # Replace 0s with 1s to avoid division by zero
        eeg /= max_val  # Normalize
        #eeg = eeg[::8]  # Downsample
        data.append(eeg)

    # Convert the list of numpy arrays to a single JAX array
    return jnp.array(data)

# Construct kernel in chunks function

A separate kernel matrix function to create test matrix. A fucntion to create the kernel matrix over the train data.

In [4]:
import time as time
def construct_kernel_matrix(df1, df2, sigma, signature_kernel, num_batches = 10):
    n = df1.shape[0]
    m = df2.shape[0]
    #initialise the kernel matrix to construct chunk by chunk
    global_kernel_matrix = jnp.zeros((n,m))
    batch_size = math.ceil(n / num_batches)
    #print(batch_size)
    # initialize signature PDE kernel
#     signature_kernel = SigKernel(refinement_factor=1, static_kernel_kind="rbf", scales = jnp.array([sigma]), add_time=True)
    for i in range(num_batches):
        # load batch i
        t1 = time.time()
        #load the first batch, to be applied to every batch
        X_batch = load_data_batch(batch_size, i)
        #X_batch = X_batch[:,::8,:]
        t2 = time.time()
        print(f"load one batch time: {t2-t1}")
        for j in tqdm(range(i, num_batches)):  # Use range(i, num_batches) to avoid redundant calculations
            # Load batch j, to be applied to X_batch
            Y_batch = load_data_batch(batch_size, j, df=df2)
            # slice and scale
            #Y_batch = Y_batch[:,::8,:]
#             X_batch *= 0.001
#             Y_batch *= 0.001
            t1 = time.time()
            # Compute the kernel matrix between these two batches
            local_kernel_matrix = signature_kernel.kernel_matrix(X_batch, Y_batch)[...,0]
            t2 = time.time()
            print(f"local kernel time: {t2-t1}")
            #Insert the local kernel matrix into the global matrix
            start_i, end_i = i * batch_size, (i + 1) * batch_size
            start_j, end_j = j * batch_size, (j + 1) * batch_size
            t1 = time.time()
            global_kernel_matrix = global_kernel_matrix.at[start_i:end_i, start_j:end_j].set(local_kernel_matrix)
            t2 = time.time()
            #print(f"assign to global kernel time: {t2-t1}")
            # For the non-diagonal blocks, where i != j, fill the symmetric position in the matrix
            if i != j:
                global_kernel_matrix = global_kernel_matrix.at[start_j:end_j, start_i:end_i].set(local_kernel_matrix.T)
    del(X_batch)
    del(Y_batch)
    return jnp.array(global_kernel_matrix)

In [5]:
def test_matrix(df1, df2, sigma, signature_kernel, batch_size=10):
    n = df1.shape[0]
    m = df2.shape[0]
    global_kernel_matrix = jnp.zeros((n, m))
    
    # Calculate the number of batches for each dataframe
    num_batches_df1 = math.ceil(n / batch_size)
    num_batches_df2 = math.ceil(m / batch_size)
    
    for i in tqdm(range(num_batches_df1), desc="Computing kernel matrix", leave=False):
        # Dynamically compute start and end indices for df1 batches
        start_i = i * batch_size
        end_i = min(start_i + batch_size, n)
        X_batch = load_data_batch(end_i - start_i, i, df=df1)

        for j in range(num_batches_df2):
            # Dynamically compute start and end indices for df2 batches
            start_j = j * batch_size
            end_j = min(start_j + batch_size, m)
            Y_batch = load_data_batch(end_j - start_j, j, df=df2)
            
            # Compute the kernel matrix between X_batch and Y_batch
            local_kernel_matrix = signature_kernel.kernel_matrix(X_batch, Y_batch)[..., 0]
            
            # Insert the local kernel matrix into the appropriate slice of the global matrix
            global_kernel_matrix = global_kernel_matrix.at[start_i:end_i, start_j:end_j].set(local_kernel_matrix)
    del(X_batch)
    del(Y_batch)
    return global_kernel_matrix

# Kernel ridge

In [6]:
def kld(p, q):
    """
    Calculate the Kullback-Leibler Divergence between two probability distributions.
    
    Parameters:
    - p: The true probability distribution (as a NumPy array).
    - q: The predicted probability distribution (as a NumPy array).
    
    Returns:
    - The KL Divergence value.
    """
    # Ensure the distributions sum to 1 (if they don't already)
    p_normalized = p / np.sum(p)
    q_normalized = q / np.sum(q)
    
    # Only consider non-zero elements
    nonzero_indices = (p_normalized > 0) & (q_normalized > 0)
    
    return np.sum(p_normalized[nonzero_indices] * np.log(p_normalized[nonzero_indices] / q_normalized[nonzero_indices]))


# Grid searching

Each sigma produced a similar result, keyboard interruption was due to GPU limit exceeded. A better grid search may have included more range.

In [28]:
#sigmas = [ 1e-2, 2.5e-2, 5e-2, 7.5e-2, 1e-1, 2.5e-1, 5e-1, 7.5e-1, 1., 2., 5., 10.]
sigmas = [1e-3, 5e-3, 1e-2, 2.5e-2, 5e-2, 7.5e-2, 1e-1] # grid search parameters
#sigmas = [7.5e-2, 1e-1, 5e-1, 1., 5., 10.]
#sigma = .1
for sigma in sigmas:
    signature_kernel = SigKernel(refinement_factor=1, static_kernel_kind="linear", scales = jnp.array([sigma]), add_time=True)
    df_train = df_unique.iloc[:600]
    train_len = len(df_train)
    v_train = v[:600]
    df_test = df_unique.iloc[2:502:5]
    test_len = len(df_test)
    v_test = v[2:502:5]
    #K_train = construct_kernel_matrix(df_train, df_train, sigma, signature_kernel)
    X_test = load_data_batch(len(df_test),0)
    X_train = load_data_batch(len(df_train),0)
    K_train = signature_kernel.kernel_matrix(X_train, X_train).reshape(train_len,train_len)
    #print(f"Computed K_train. Shape: {K_train.shape}")
    K_test = signature_kernel.kernel_matrix(X_train, X_test).reshape(test_len,train_len)
    #print(f"Computed K_test. Shape: {K_test.shape}")
    krr = KernelRidge(kernel='precomputed', alpha=1.5)
    #print("Initialised model")
    krr.fit(K_train,v_train)
    #print("Trained model")
    train_predictions = krr.predict(K_train)
    #print("Computed train predictions")
    test_predictions = krr.predict(K_test)
    #print("Computed test predictions")
    train_mse = mean_squared_error(train_predictions, v_train)
    test_mse = mean_squared_error(test_predictions, v_test)
    test_kld = kld(v_test, (jnn.relu(test_predictions).T/jnn.relu(test_predictions).sum(axis=1)).T)
    print(f"Sigma: {sigma} ---- Train error: {train_mse: .4f} ---- Test error: {test_mse: .4f} ---- Test KLDiv: {test_kld: .4f}")
    #print(test_predictions)

Sigma: 0.001 ---- Train error:  0.0564 ---- Test error:  0.0614 ---- Test KLDiv:  0.8953
Sigma: 0.005 ---- Train error:  0.0564 ---- Test error:  0.0614 ---- Test KLDiv:  0.8953


KeyboardInterrupt: 

In [9]:
sigma = 0.01 # define best sigma
# initialise signature kernel
signature_kernel = SigKernel(refinement_factor=2, static_kernel_kind="linear", scales = jnp.array([sigma]), add_time=True)
#create train set
df_train = df_unique.iloc[:4000]
train_len = len(df_train)
v_train = v[:4000]
#define a test set
df_test = df_unique.iloc[9000:]
test_len = len(df_test)
v_test = v[9000:]

K_train = construct_kernel_matrix(df_train, df_train, sigma, signature_kernel, 10)
krr = KernelRidge(kernel='precomputed', alpha=1.5)
krr.fit(K_train,v_train)
train_predictions = krr.predict(K_train)
del(K_train)

X_test = load_data_batch(len(df_test),0)
X_train = load_data_batch(len(df_train),0)
K_test = signature_kernel.kernel_matrix(X_train, X_test).reshape(test_len,train_len)

K_test = test_matrix(df_test, df_train, sigma, signature_kernel, 1000)
test_predictions = krr.predict(K_test)
# print the exact difference between model output and targets, then KLDiv defined on our probability approximate
train_mse = mean_squared_error(train_predictions, v_train)
test_mse = mean_squared_error(test_predictions, v_test)
test_kld = kld(v_test, (jnn.relu(test_predictions).T/jnn.relu(test_predictions).sum(axis=1)).T)
print(f"Sigma: {sigma} ---- Train error: {train_mse: .4f} ---- Test error: {test_mse: .4f} ---- Test KLDiv: {test_kld}")

load one batch time: 3.0408051013946533


  0%|          | 0/10 [00:00<?, ?it/s]

local kernel time: 25.892420530319214


 10%|█         | 1/10 [00:54<08:10, 54.48s/it]

local kernel time: 24.568979501724243


...

100%|██████████| 2/2 [01:37<00:00, 48.80s/it]


load one batch time: 3.480668783187866


  0%|          | 0/1 [00:00<?, ?it/s]

local kernel time: 41.886030197143555


100%|██████████| 1/1 [00:46<00:00, 46.03s/it]


In [ ]:
sigma = 0.01
signature_kernel = SigKernel(refinement_factor=1, static_kernel_kind="linear", scales = jnp.array([sigma]), add_time=True)

K_train = construct_kernel_matrix(df_unique, df_unique, sigma, signature_kernel)
# Calculate result on the actual test data
eeg_test = pq.read_table('/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/3911565283.parquet').to_pandas()
eeg_test = eeg_test.loc[:, eeg_test.columns.isin(FEATS)].iloc[4000:6000]
eeg_test = eeg_test.ffill().bfill()
eeg_test = eeg_test.values
eeg_test /= np.abs(eeg_test).max(axis=0)
eeg_test = eeg_test.reshape(1,eeg_test.shape[0],-1)[:,::8,:]
X_train = load_data_batch(len(df_unique),0)
K_test = signature_kernel.kernel_matrix(X_train, eeg_test).reshape(1, len(df_unique))
del(X_train)
#print(f"Computed K_test. Shape: {K_test.shape}")
krr = KernelRidge(kernel='precomputed', alpha=1.5)
#print("Initialised model")
krr.fit(K_train,v)
#print("Trained model")
train_predictions = krr.predict(K_train)
#print("Computed train predictions")
test_predictions = krr.predict(K_test)
#print("Computed test predictions")
train_mse = mean_squared_error(train_predictions, v)
print(f"Sigma: {sigma} ---- Train error: {train_mse: .4f} ---- Test predictions: {test_predictions}")

In [10]:
from joblib import dump, load

# save model
model_path = '/kaggle/working/kernel_ridge_model_fft_filter_final_model.joblib'
dump(krr, model_path)

['/kaggle/working/kernel_ridge_model_fft_filter_final_model.joblib']

In [8]:
from joblib import dump, load
model_path = '/kaggle/working/kernel_ridge_model_fft_filter_final_model.joblib'
krr_load = load(model_path)
#load model

In [38]:
# Check the test probabilities against the actual test probabilities
test_probabilities_scale = (jnn.relu(test_predictions).T/jnn.relu(test_predictions).sum(axis = 1)).T
#print(jnn.relu(test_predictions).sum(axis=1))
i = 17
print(f"predicted: {test_probabilities_scale[i]}")
print(f"actual: {v_test[i]}")


predicted: [0.05232895 0.18702224 0.12539278 0.08286361 0.0868229  0.46556962]
actual: [0.         0.63636364 0.09090909 0.09090909 0.         0.18181818]


In [ ]:
#Update the submission table
submission = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv')
labels=['seizure','lpd','gpd','lrda','grda','other']
for i in range(len(labels)):
    submission[f'{labels[i]}_vote']=test_probabilities_scale[:,i]
submission.to_csv("submission.csv",index=None)
display(submission.head())